# 문제 유형1


* 다음은 음식 조리 시간 관련 데이터셋이다. 이 데이터셋을 활용하여 음식 조리 시간에 대한 예측을 수행하시오.


* 파일명: indian_food2.csv (구분자: 쉼표(,), 255 행, 10 컬럼, UTF-8 Encoding, 결측치: Empty string)


In [1]:
# 패키지 로딩

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
dat=pd.read_csv('data/indian_food2.csv', na_values=['nan', 'NA',' ','']) # na_values=[' ','']
print(dat.columns)
print(dat.dtypes)

Index(['id', 'name', 'ingredients', 'diet', 'prep_time', 'cook_time',
       'flavor_profile', 'course', 'state', 'region'],
      dtype='object')
id                 int64
name              object
ingredients       object
diet              object
prep_time          int64
cook_time          int64
flavor_profile    object
course            object
state             object
region            object
dtype: object


* 다음의 전처리를 먼저 수행한 후 분석 진행하시오.

In [3]:
# 단계 1-1. 지역(state) 컬럼을 제거하시오.

pre1=dat.drop(columns='state')

In [4]:
# 단계 1-2. 조리 시간(cook_time)과 재료 준비 시간(prep_time)이 양수(>0)인 데이터만 추출하시오.

pre2=pre1[(pre1['cook_time'] > 0)]
pre2=pre2[(pre2['prep_time'] > 0)]
len(pre2)

225

In [5]:
# 단계 1-3. region의 값이  “-1”인 경우와 빈문자(empty string)는 제거한 후 ‘North East’ 지역을 ‘North’로 변경하시오.

pre3=pre2[~(pre2.region.isin(['-1','', ' ']))]
pre3=pre3.dropna()
pre3.loc[pre3.region=='North East', 'region']='North'
len(pre3)

212

In [6]:
# 단계 1-4. 음식 맛 구분(flavor_profile)을 다음과 같이 2가지로 정리한 변수(변수명: ed_flavor_profile)를 추가하시오.
# IF flavor_profile= ‘sweet’ then ‘sweet’ else ‘not sweet’

pre4=pre3.copy()

pre4['ed_flavor_profile']=np.where(pre4.flavor_profile == 'sweet', 'sweet', 'not sweet')


In [7]:
# 단계 1-5. 음식 종류(course)를 2가지로 정리한 변수(변수명: ed_course)를 추가하시오.
# IF course = ‘main course ‘ then ‘main course’ else ‘not main course’
pre4['ed_course']=np.where(pre4.course == 'main course', 'main course', 'not main course')

In [8]:
# 단계 1-6. 재료 목록(ingredients)를 이용하여 재료 수 변수(변수명: ingredients_no)를 추가하시오.
#  예: 고추장, 마늘  ingredients_no=2
pre4['ingredients_no']=pre4['ingredients'].apply(lambda x: len(str(x).split(',')))
print(pre4['ingredients'].head(5))
print(pre4['ingredients_no'].head(5))

basetable1=pre4.copy()

0                      Maida flour, yogurt, oil, sugar
1                              Gram flour, ghee, sugar
2         Carrots, milk, sugar, ghee, cashews, raisins
3    Flour, ghee, kewra, milk, clarified butter, su...
4    Milk powder, plain flour, baking powder, ghee,...
Name: ingredients, dtype: object
0     4
1     3
2     6
3    10
4     8
Name: ingredients_no, dtype: int64


In [9]:
len(basetable1)

212

* 상기 전처리를 완료한 데이터 프레임(데이터 프레임명: basetable1, 212 Rows)으로 다음 분석(문제 1~3) 수행

1. (basetable1을 이용) 2범주로 정리된 음식 맛(ed_flavor_profile)과 5개로 정리된 음식 지역(region) 간에 통계적으로 유의미한 차이가 있는지 적절한 검정을 수행하고, 검정 결과 중 검정 통계량을 기술하시오.  
- 검정 통계량은 소수점 넷째 자리 이하는 버리고 소수점 셋째 자리까지 기술.  
(답안 예시) 0.123

In [10]:
from scipy.stats import chi2_contingency

Q1=pd.crosstab(index=basetable1.ed_flavor_profile, columns=basetable1.region)

Q1_out=chi2_contingency(Q1)

print(Q1_out)

Q1_ans=Q1_out[0]

print('\n Q1 정답: ', Q1_ans )  # 22.689047335851807

(22.689047335851807, 0.0001460865752415348, 4, array([[ 1.95283019, 17.5754717 , 39.70754717, 35.80188679, 42.96226415],
       [ 1.04716981,  9.4245283 , 21.29245283, 19.19811321, 23.03773585]]))

 Q1 정답:  22.689047335851807


2. (basetable1을 이용) 5개 지역(region)에 따라 재료 준비 시간(prep_time)의 평균이 통계적으로 유의미한 차이가 있는지 적절한 검정을 수행하고, 검정 결과 중 유의 확률(P-Value) 값을 기술하시오.

- 주어진 데이터는 등분산성을 가정하며, 유의 확률(P-Value) 값은 소수점 넷째 자리 이하는 버리고 소수점 셋째 자리까지 기술.  
(답안 예시) 0.123

In [11]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

Q2=ols('prep_time~C(region)', basetable1).fit()
Q2_out=anova_lm(Q2)

print(Q2_out)

Q2_ans=Q2_out['PR(>F)'][0]

print('\n Q2 정답: ', Q2_ans)  # 0.06442857691647358

              df        sum_sq       mean_sq         F    PR(>F)
C(region)    4.0  5.409888e+04  13524.720512  2.254672  0.064429
Residual   207.0  1.241696e+06   5998.531150       NaN       NaN

 Q2 정답:  0.06442857691647358


3. (basetable1을 이용) 음식 조리 시간(cook_time)을 예측하는 모델을 생성하고자 한다. 이를 위하여 basetable1에 다음의 전처리를 추가로 수행하시오.  


단계 1-7. 다음 변수들을 추가하시오(총 2개).  

- ed_course으로부터 생성한 가변수(1개), ed_flavor_profile로부터 생성한 가변수(1개)


In [12]:
step7=pd.get_dummies(basetable1, 
                     columns=['ed_course', 'ed_flavor_profile'], 
                     drop_first=True)

print(step7.columns)

Index(['id', 'name', 'ingredients', 'diet', 'prep_time', 'cook_time',
       'flavor_profile', 'course', 'region', 'ingredients_no',
       'ed_course_not main course', 'ed_flavor_profile_sweet'],
      dtype='object')


단계 1-8. 음식 조리 시간(cook_time) 예측을 위한 선형 회귀 모형을 생성하고자 한다. 다음 가이드에 따라 Train Data로 학습시킨 모형을 Test Data에 적용하여 음식 조리 시간(cook_time)을 예측하시오.  

- Train Data: id가 3의 배수가 아닌 데이터, Test Data: id가 3의 배수인 데이터  
- 독립변수(3개): 단계 1-7의 가변수들(2개), prep_time  


In [13]:
train=step7[(step7.id % 3) != 0]
test=step7[(step7.id % 3) == 0]

x_list=['ed_course_not main course', 'ed_flavor_profile_sweet', 'prep_time']

In [14]:
from sklearn.linear_model import LinearRegression

lm=LinearRegression().fit(train[x_list], train['cook_time'])

Q3_out=lm.predict(test[x_list])

In [15]:
from statsmodels.api import OLS, add_constant

train1=add_constant(train[x_list])
test1=add_constant(test[x_list])

lm1=OLS(train['cook_time'], train1).fit()
Q3_out=lm1.predict(test1)

생성한 회귀 분석 모델을 Test Data에 적용하여 음식 조리 시간(cook_time)를 예측한 후
예측값들에 대한 평균제곱근오차(Root Mean Square Error, RMSE)를 기술하시오.  

- RMSE는 소수점 둘째 자리 이하는 버리고 소수점 첫째 자리까지 기술. (답안 예시) 1.2


In [16]:
from sklearn.metrics import mean_squared_error

Q3_ans=np.sqrt(mean_squared_error(test['cook_time'], Q3_out))
print('\n Q3 정답: ', Q3_ans)  # 82.72581919372088


 Q3 정답:  83.29102970312529
